`GROUPING SETS` 子句在查询中生成多个分组集。

In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from E:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


In [3]:
sql = """
DROP TABLE IF EXISTS sales;

CREATE TABLE sales (
    brand VARCHAR NOT NULL,
    segment VARCHAR NOT NULL,
    quantity INT NOT NULL,
    PRIMARY KEY (brand, segment)
);

INSERT INTO sales (brand, segment, quantity)
VALUES
    ('ABC', 'Premium', 100),
    ('ABC', 'Basic', 200),
    ('XYZ', 'Premium', 100),
    ('XYZ', 'Basic', 300);
"""

cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x2c1ed321a60>

In [4]:
sql = """
SELECT * FROM sales
"""

run_sql(cursor, sql)

  brand  segment  quantity
0   ABC  Premium       100
1   ABC    Basic       200
2   XYZ  Premium       100
3   XYZ    Basic       300


In [5]:
sql = """
SELECT
    brand,
    segment,
    SUM (quantity)
FROM
    sales
GROUP BY
    brand,
    segment;
"""

run_sql(cursor, sql)

  brand  segment  sum
0   XYZ    Basic  300
1   ABC  Premium  100
2   ABC    Basic  200
3   XYZ  Premium  100


In [6]:
sql = """
SELECT
    brand,
    SUM (quantity)
FROM
    sales
GROUP BY
    brand;
"""

run_sql(cursor, sql)

  brand  sum
0   ABC  300
1   XYZ  400


In [7]:
sql = """
SELECT
    brand,
    segment,
    SUM (quantity)
FROM
    sales
GROUP BY
    brand,
    segment

UNION ALL

SELECT
    brand,
    NULL,
    SUM (quantity)
FROM
    sales
GROUP BY
    brand

UNION ALL

SELECT
    NULL,
    segment,
    SUM (quantity)
FROM
    sales
GROUP BY
    segment

UNION ALL

SELECT
    NULL,
    NULL,
    SUM (quantity)
FROM
    sales;
"""

run_sql(cursor, sql)

  brand  segment  sum
0   XYZ    Basic  300
1   ABC  Premium  100
2   ABC    Basic  200
3   XYZ  Premium  100
4   ABC     None  300
5   XYZ     None  400
6  None    Basic  500
7  None  Premium  200
8  None     None  700


对于上述的查询，可以试用更高效的 `GROUPING SETS`。
```
SELECT
    c1,
    c2,
    aggregate_function(c3)
FROM
    table_name
GROUP BY
    GROUPING SETS (
        (c1, c2),
        (c1),
        (c2),
        ()
);
```

In [8]:
sql = """
SELECT
    brand,
    segment,
    SUM (quantity)
FROM
    sales
GROUP BY
    GROUPING SETS (
        (brand, segment),
        (brand),
        (segment),
        ()
    );
"""

run_sql(cursor, sql)

  brand  segment  sum
0  None     None  700
1   XYZ    Basic  300
2   ABC  Premium  100
3   ABC    Basic  200
4   XYZ  Premium  100
5   ABC     None  300
6   XYZ     None  400
7  None    Basic  500
8  None  Premium  200


GROUPING 函数
```
GROUPING( column_name | expression)
```
`column_name` 或 `expression` 必须与 `GROUP BY` 子句中指定的相匹配。如果参数是当前分组集的成员，则 `GROUPING()` 函数返回位 0，否则返回 1。

In [9]:
# GROUPING 函数
sql = """
SELECT
	GROUPING(brand) grouping_brand,
	GROUPING(segment) grouping_segment,
	brand,
	segment,
	SUM (quantity)
FROM
	sales
GROUP BY
	GROUPING SETS (
		(brand),
		(segment),
		()
	)
ORDER BY
	brand,
	segment;
"""

run_sql(cursor, sql)

"""
当 grouping_brand 中的值为0时，sum列显示 brad 的小计。
当 grouping_segment 中的值为0时，sum列显示 segment 的小计。
"""

   grouping_brand  grouping_segment brand  segment  sum
0               0                 1   ABC     None  300
1               0                 1   XYZ     None  400
2               1                 0  None    Basic  500
3               1                 0  None  Premium  200
4               1                 1  None     None  700


In [10]:
sql = """
SELECT
	GROUPING(brand) grouping_brand,
	GROUPING(segment) grouping_segment,
	brand,
	segment,
	SUM (quantity)
FROM
	sales
GROUP BY
	GROUPING SETS (
		(brand),
		(segment),
		()
	)
HAVING GROUPING(brand) = 0	
ORDER BY
	brand,
	segment;
"""

run_sql(cursor, sql)

   grouping_brand  grouping_segment brand segment  sum
0               0                 1   ABC    None  300
1               0                 1   XYZ    None  400
